In [ ]:
!pip install pdfplumber

In [ ]:
import os
import pdfplumber
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pdf_folder = '/content/drive/MyDrive/pdf'

In [ ]:
def extract_entities(text):
    names = re.findall(r"(\b[A-Z][a-z]+ [A-Z][a-z]+\b)", text)
    phones = re.findall(r"(\b\d{10}\b)", text)
    emails = re.findall(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", text)
    extracted_labels = ['O'] * len(text.split())
    for name in names:
        name_tokens = name.split()
        name_start = text.index(name_tokens[0])
        name_end = name_start + len(name_tokens)
        extracted_labels[name_start] = 'B-NAME'
        for i in range(name_start + 1, name_end):
            extracted_labels[i] = 'I-NAME'
    return extracted_labels

In [ ]:
for pdf_number in range(1, 1001):
  pdf_filename = f"{pdf_number}.pdf"
  pdf_path = os.path.join(pdf_folder, pdf_filename)
  if os.path.exists(pdf_path):
    try:
      with pdfplumber.open(pdf_path) as pdf:
        pdf_text = ""
        for page in pdf.pages:
          pdf_text += page.extract_text()
        extracted_labels = extract_entities(pdf_text)
        texts.append(pdf_text)
        labels.append(extracted_labels)
    except Exception as e:
      continue

In [ ]:
def clean_text(text):
  text = text.replace("\n", " ").strip()
  return text

def preprocess_data(texts, labels, label_map):
  processed_texts = [clean_text(text) for text in texts]
  return processed_texts

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
  pdf_text = ""
  for page in pdf.pages:
    pdf_text += page.extract_text()
  preprocessed_pdf_text = clean_text(pdf_text)
try:
  openai.api_type = "azure"
  openai.api_base = "<endpoint url>"
  openai.api_version = "<api version>"
  openai.api_key = "<api key>"
  response = openai.Completion.create(engine="res", prompt=f"{preprocessed_pdf_text}", temperature=1.0, max_tokens=350, top_p=1, frequency_penalty=0, presence_penalty=0, stop=None)
  summary = response.choices[0].text.strip()
  print(summary)
except openai.error.OpenAIError as e:
    print(f"OpenAI API Error: {e}")

NameError: ignored